In [1]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from utilsCPD import *
import matplotlib.pyplot as plt
from SWCPD import BaseDetector as SWDCP


BATCH_SIZE  =32
mnist_transforms = transforms.Compose([transforms.ToTensor()])
train_val_dataset = datasets.MNIST(root="./datasets/", train=True, download=True, transform=mnist_transforms)
test_dataset = datasets.MNIST(root="./datasets/", train=False, download=True, transform=mnist_transforms)
train_size = int(0.9 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset=train_val_dataset, lengths=[train_size, val_size])
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [2]:
all_batches = []
all_labels =[]
torch.manual_seed(10)
for x,y in test_dataloader:
    bs = x.shape[0]
    all_batches.append(x.flatten().reshape(bs,784))
    all_labels.append(y)

all_batches = torch.cat(all_batches,dim=0)
all_labels = torch.cat(all_labels,dim=0)

print(all_batches.shape)

torch.Size([10000, 784])


In [3]:
Labels_Sampled = []
seed = 2025
rng = np.random.default_rng(seed)


def sample_classes(seed,n_classes = 3,nb_experiments=5):
    rng = np.random.default_rng(seed)
    Labels_Sampled = []
    for run in range(nb_experiments):
        l = tuple(rng.choice(np.arange(10),n_classes,replace=False))
        if l not in Labels_Sampled:
            Labels_Sampled.append(l)
    return Labels_Sampled

Labels_Sampled = sample_classes(seed=seed, n_classes=3,nb_experiments=5)

print(Labels_Sampled)
def FilterData(all_batches,all_labels,lables,N_Samples=200):
    data = []
    cps = []
    for l in lables:
        d =all_batches[all_labels==l].detach().numpy()
        data.append(d[:N_Samples])
        cps.append(len(d[:N_Samples]))
    return np.vstack(data),np.cumsum(cps)[:-1]


FilterData(all_batches,all_labels,list(Labels_Sampled[0]))


[(3, 9, 8), (6, 5, 8), (0, 4, 3), (6, 7, 8), (4, 3, 0)]


(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([200, 400]))

In [64]:
LABELS_Sampled = [sample_classes(seed=seed,n_classes=3,nb_experiments=5),sample_classes(seed=seed, n_classes=4,nb_experiments=5),sample_classes(seed=seed, n_classes=5,nb_experiments=5)]
LABELS_Sampled

[[(3, 9, 8), (6, 5, 8), (0, 4, 3), (6, 7, 8), (4, 3, 0)],
 [(3, 7, 8, 9), (8, 3, 6, 5), (7, 6, 3, 8), (8, 7, 3, 2), (6, 7, 4, 2)],
 [(2, 6, 3, 7, 9),
  (5, 2, 3, 4, 0),
  (4, 0, 3, 8, 2),
  (4, 6, 7, 0, 2),
  (0, 1, 9, 3, 2)]]

In [23]:
METRICS_SWD = {}
Data_OUT = {}
LABELS_Sampled = [sample_classes(seed=seed,n_classes=3,nb_experiments=5),sample_classes(seed=seed, n_classes=4,nb_experiments=5),sample_classes(seed=seed, n_classes=5,nb_experiments=5)]
ind = 0
ind2 = 0
LABELS_Sampled
for id,i in enumerate(range(15)):
    print(id)
    Data_OUT[id] = {}
    METRICS_SWD[id]  = {}
    if i in [5,10]:
        ind+=1
        ind2 = 0
    df, GroundTruth = FilterData(all_batches,all_labels,list(LABELS_Sampled[ind][ind2]),N_Samples=200)
    Data_OUT[id]['data'] = df
    Data_OUT[id]['target'] = GroundTruth
    ind2+=1
    print(GroundTruth)
    detector = SWDCP(data=df,window_length=50,max_history=50,significance=0.05)
    print(detector.device)
    detector.process_dataloader(p=4,n_theta=5000,seed=2025)
    f1, covering_score, auc_score = detector.evaluate(GroundTruth,20)
    METRICS_SWD[id]['F1'] = f1
    METRICS_SWD[id]['Covering'] = covering_score
    METRICS_SWD[id]['AUC'] = auc_score
    print(METRICS_SWD[id])

0
[200 400]
cuda


 31%|███▏      | 173/551 [00:04<00:09, 38.67it/s]

Change detected at: 216 
Initiate new segment


 68%|██████▊   | 376/551 [00:09<00:04, 36.78it/s]

Change detected at: 420 
Initiate new segment


...

Change detected at: 615 
Initiate new segment


 81%|████████  | 770/951 [00:17<00:04, 43.64it/s]

Change detected at: 815 
Initiate new segment


100%|██████████| 951/951 [00:22<00:00, 42.92it/s]

F1 score: 1.0
Covering: 0.9070618141548373
1.0
{'F1': 1.0, 'Covering': 0.9070618141548373, 'AUC': 1.0}


In [24]:
f1,cov, AUC = [],[],[]
for k,metrics in METRICS_SWD.items():
    f1.append(metrics['F1'])
    cov.append(metrics['Covering'])
    AUC.append(metrics['AUC'])


#print(np.mean(f1))
print(np.mean(cov))
print(np.mean(AUC))

0.8664722080501247
0.9033333333333334


In [ ]:
# import json

# class NumpyEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return super().default(obj)
    
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(Data_OUT, f, ensure_ascii=False, indent=4,cls=NumpyEncoder)

In [ ]:
from claspy.segmentation import BinaryClaSPSegmentation
ind = 0
ind2 = 0
METRICS_Claspy = {}
for id,i in enumerate(range(15)):
    print(id)
    METRICS_Claspy[id]  = {}
    if i in [5,10]:
        ind+=1
        ind2 = 0
    df, GroundTruth = FilterData(all_batches,all_labels,list(LABELS_Sampled[ind][ind2]),N_Samples=200)
    ind2+=1
    print(GroundTruth)
    clasp = BinaryClaSPSegmentation(window_size=100).fit(df.astype(np.float64))
    f1,auc_score = f_measure({'0':GroundTruth},clasp.change_points,20)
    covering_score = covering({'0':GroundTruth},clasp.change_points,df.shape[0])
    METRICS_Claspy[id]['F1'] = f1
    METRICS_Claspy[id]['Covering'] = covering_score
    METRICS_Claspy[id]['AUC'] = auc_score
    print(METRICS_Claspy[id])

In [54]:
f1,cov, AUC = [],[],[]
for k,metrics in METRICS_Claspy.items():
    f1.append(metrics['F1'])
    cov.append(metrics['Covering'])
    AUC.append(metrics['AUC'])
print(np.std(cov))
print(np.std(AUC))

0.05499719409228699
0.027498597046143512


In [21]:
data_out = {}
ind2=0
ind=0
for id,i in enumerate(range(15)):
    if i in [5,10]:
        ind+=1
        ind2 = 0
    df, GroundTruth = FilterData(all_batches,all_labels,list(LABELS_Sampled[ind][ind2]),N_Samples=200)
    data_out[id] = {'data':df,'labels':GroundTruth}

In [22]:
# import json

# class NumpyEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return super().default(obj)
    
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(data_out, f, ensure_ascii=False, indent=4,cls=NumpyEncoder)

### Results for E-divisive and BOCPD

In [48]:
import json

with open("./R/res//MNIST_Edivise.json") as f:
    ECP_Results = json.load(f)


F1,Covering, AUC = [],[],[]

for id, cps in ECP_Results.items():
    ground_truths = data_out[int(id)]['labels']
    n = ground_truths[-1] + 200
    cps = cps[1:-1]

    f1, AUC_score = f_measure({'0':ground_truths},cps,20)
    covering_score = covering({'0':ground_truths},cps,n)
    F1.append(f1)
    AUC.append(AUC_score)
    Covering.append(covering_score)

print(np.mean(F1))
print(np.std(Covering))
print(np.std(AUC))

0.9569023569023568
0.05164545795524169
0.047790695928014576


In [51]:
with open("./R/res/MNIST_BOCPD.json") as f:
    ECP_Results = json.load(f)


F1,Covering, AUC = [],[],[]

for id, cps in ECP_Results.items():
    #print(cps)
    ground_truths = data_out[int(id)]['labels']
    n = ground_truths[-1] + 200
    cps = cps[0][1:]
    #print(cps)
    f1, AUC_score = f_measure({'0':ground_truths},cps,20)
    covering_score = covering({'0':ground_truths},cps,n)
    F1.append(f1)
    AUC.append(AUC_score)
    #print(AUC_score)
    Covering.append(covering_score)

print(np.mean(F1))
print(np.mean(Covering))
print(np.mean(AUC))

0.6844444444444445
0.7847463171351448
0.6916666666666668
